# **Лабораторная работа №6:**
## "Разработка системы предсказания поведения на основании графовых моделей"

---

*Цель*: обучение работе с графовым типом данных и графовыми нейронными сетями.

*Задача*: подготовить графовый датасет из базы данных о покупках и построить модель предсказания совершения покупки.

---

## Графовые нейронные сети

**Графовые нейронные сети** - тип нейронной сети, которая напрямую работает со структурой графа. Типичным применениями GNN являются:
- Классификация узлов;
- Предсказание связей;
- Графовая классификация;
- Распознавание движений;
- Рекомендательные системы.

В данной лабораторной работе будет происходить работа над **графовыми сверточными сетями**. Отличаются они от сверточных нейронных сетей нефиксированной структурой, функция свертки не является .

Подробнее можно прочитать тут: https://towardsdatascience.com/understanding-graph-convolutional-networks-for-node-classification-a2bfdb7aba7b

Тут можно почитать современные подходы к использованию графовых сверточных сетей 
https://paperswithcode.com/method/gcn

---

## Датасет
В качестве базы данных предлагаем использовать датасет о покупках пользователей в одном магазине товаров RecSys Challenge 2015 (https://www.kaggle.com/datasets/chadgostopp/recsys-challenge-2015). 

Скачать датасет можно отсюда: https://drive.google.com/drive/folders/1gtAeXPTj-c0RwVOKreMrZ3bfSmCwl2y-?usp=sharing
(lite-версия является облегченной версией исходного датасета, рекомендуем использовать её)

Также рекомендуем загружать данные в виде архива и распаковывать через пакет zipfile или/и скачивать датасет в собственный Google Drive и примонтировать его в колаб.

---

### Установка библиотек, выгрузка исходных датасетов

In [1]:
# Slow method of installing pytorch geometric
# !pip install torch_geometric
# !pip install torch_sparse
# !pip install torch_scatter

# Install pytorch geometric
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.11.0%2Bcu113.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.11.0%2Bcu113.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.11.0%2Bcu113.html
!pip install torch-geometric -f https://pytorch-geometric.com/whl/torch-1.11.0%2Bcu113.html
!pip install torch-scatter==2.0.8 -f https://data.pyg.org/whl/torch-1.11.0%2Bcu113.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0%2Bcu113.html
     |████████████████████████████████| 3.5 MB 9.8 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0%2Bcu113.html
     |████████████████████████████████| 2.5 MB 7.1 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0%2Bcu113.html
     |████████████████████████████████| 750 kB 9.7 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0%2Bcu113.html
     |████████████████████████████████| 407 kB 10.0 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=caa368a09423372f9eb35d1ec5dba806ed4f18ce2d13d7a40cea27a3204f411c
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric
Looking in links: https://data.pyg.org/whl/torch-1.11.0%2Bcu113.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.8-cp37-cp37m-linu

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import numpy as np
import pandas as pd
import pickle
import csv
import os

from sklearn.preprocessing import LabelEncoder

import torch

# PyG - PyTorch Geometric
from torch_geometric.data import Data, DataLoader, InMemoryDataset

from tqdm import tqdm


RANDOM_SEED = 42 #@param { type: "integer" }
BASE_DIR = '/content/' #@param { type: "string" }
np.random.seed(RANDOM_SEED) 

In [8]:
# Check if CUDA is available for colab
torch.cuda.is_available

<function torch.cuda.is_available>

In [14]:
# Unpack files from zip-file
import zipfile
with zipfile.ZipFile(BASE_DIR + 'yoochoose-data-lite.zip', 'r') as zip_ref:
    zip_ref.extractall(BASE_DIR)

### Анализ исходных данных

In [15]:
# Read dataset of items in store
df = pd.read_csv(BASE_DIR + 'yoochoose-clicks-lite.dat')
# df.columns = ['session_id', 'timestamp', 'item_id', 'category'] 
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,session_id,timestamp,item_id,category
0,9,2014-04-06T11:26:24.127Z,214576500,0
1,9,2014-04-06T11:28:54.654Z,214576500,0
2,9,2014-04-06T11:29:13.479Z,214576500,0
3,19,2014-04-01T20:52:12.357Z,214561790,0
4,19,2014-04-01T20:52:13.758Z,214561790,0


In [16]:
# Read dataset of purchases
buy_df = pd.read_csv(BASE_DIR + 'yoochoose-buys-lite.dat')
# buy_df.columns = ['session_id', 'timestamp', 'item_id', 'price', 'quantity']
buy_df.head()

,session_id,timestamp,item_id,price,quantity
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,489758,2014-04-06T09:59:52.422Z,214826955,1360,2
3,489758,2014-04-06T09:59:52.476Z,214826715,732,2
4,489758,2014-04-06T09:59:52.578Z,214827026,1046,1


In [17]:
# Filter out item session with length < 2
df['valid_session'] = df.session_id.map(df.groupby('session_id')['item_id'].size() > 2)
df = df.loc[df.valid_session].drop('valid_session',axis=1)
df.nunique()

session_id    1000000
timestamp     5557758
item_id         37644
category          275
dtype: int64

In [18]:
# Randomly sample a couple of them
NUM_SESSIONS = 50000 #@param { type: "integer" }
sampled_session_id = np.random.choice(df.session_id.unique(), NUM_SESSIONS, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id     50000
timestamp     278442
item_id        18461
category         110
dtype: int64

In [19]:
# Average length of session
df.groupby('session_id')['item_id'].size().mean()

5.56902

In [20]:
# Encode item and category id in item dataset so that ids will be in range (0,len(df.item.unique()))
item_encoder = LabelEncoder()
category_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id)
df['category']= category_encoder.fit_transform(df.category.apply(str))
df.head()

,session_id,timestamp,item_id,category
0,9,2014-04-06T11:26:24.127Z,3496,0
1,9,2014-04-06T11:28:54.654Z,3496,0
2,9,2014-04-06T11:29:13.479Z,3496,0
102,171,2014-04-03T17:45:25.575Z,10049,0
103,171,2014-04-03T17:45:33.177Z,10137,0


In [21]:
# Encode item and category id in purchase dataset
buy_df = buy_df.loc[buy_df.session_id.isin(df.session_id)]
buy_df['item_id'] = item_encoder.transform(buy_df.item_id)
buy_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,session_id,timestamp,item_id,price,quantity
46,489491,2014-04-06T12:41:34.047Z,12633,1046,4
47,489491,2014-04-06T12:41:34.091Z,12634,627,2
61,70353,2014-04-06T10:55:06.086Z,14345,41783,1
62,489671,2014-04-03T15:48:37.392Z,12489,4188,1
63,489671,2014-04-03T15:59:35.495Z,12489,4188,1


In [22]:
# Get item dictionary with grouping by session
buy_item_dict = dict(buy_df.groupby('session_id')['item_id'].apply(list))
buy_item_dict

{714: [14720, 14915, 14917, 3089],
 6016: [15154],
 9797: [12459, 11831],
 9862: [13621],
 10457: [10079, 2951],
 10587: [11764],
 10678: [6310, 3914],
 13476: [13631, 12881, 12878, 12880, 12852],
 16953: [2883, 7739],
 19029: [8276, 2171, 10385, 11419],
 19958: [10059, 10059],
 23548: [11236],
 24439: [12506, 12497],
 28709: [4037],
 29647: [12830, 12827],
 33907: [2480, 6012],
 34541: [627, 12827],
 36548: [14720, 14916],
 38019: [11222, 11227],
 38261: [3447],
 41333: [11839, 12826, 11839],
 41598: [10712, 10711, 2034, 10713],
 43834: [11203, 11203],
 44153: [15075, 15088],
 44813: [12819, 12634],
 48974: [7428, 12774],
 49886: [2373, 11207, 11221, 10625],
 54961: [1965, 11360, 12812, 7803],
 55877: [4804],
 62553: [12793],
 64802: [11317],
 69277: [7933],
 70353: [14345],
 71832: [11839, 12362],
 73271: [11236, 11236, 11237],
 74083: [12864],
 79937: [11884, 11825, 11825, 11884],
 87673: [12608],
 88198: [11201, 11201],
 88723: [11236, 11236],
 89393: [11236],
 91903: [11200],
 922

### Сборка выборки для обучения

In [23]:
# Transform df into tensor data
def transform_dataset(df, buy_item_dict):
    data_list = []

    # Group by session
    grouped = df.groupby('session_id')
    for session_id, group in tqdm(grouped):    
        le = LabelEncoder()
        sess_item_id = le.fit_transform(group.item_id)
        group = group.reset_index(drop=True)
        group['sess_item_id'] = sess_item_id

        #get input features
        node_features = group.loc[group.session_id==session_id,
                                    ['sess_item_id','item_id','category']].sort_values('sess_item_id')[['item_id','category']].drop_duplicates().values
        node_features = torch.LongTensor(node_features).unsqueeze(1)
        target_nodes = group.sess_item_id.values[1:]
        source_nodes = group.sess_item_id.values[:-1]

        edge_index = torch.tensor([source_nodes,
                                target_nodes], dtype=torch.long)
        x = node_features

        #get result
        if session_id in buy_item_dict:
            positive_indices = le.transform(buy_item_dict[session_id])
            label = np.zeros(len(node_features))
            label[positive_indices] = 1
        else:
            label = [0] * len(node_features)

        y = torch.FloatTensor(label)

        data = Data(x=x, edge_index=edge_index, y=y)

        data_list.append(data)
    
    return data_list

# Pytorch class for creating datasets
class YooChooseDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return [BASE_DIR+'yoochoose_click_binary_100000_sess.dataset']

    def download(self):
        pass
    
    def process(self):
        data_list = transform_dataset(df, buy_item_dict)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [24]:
# Prepare dataset
dataset = YooChooseDataset('./')

Processing...
100%|██████████| 50000/50000 [02:49<00:00, 295.68it/s]
Done!


### Разделение выборки

In [25]:
# train_test_split
dataset = dataset.shuffle()
one_tenth_length = int(len(dataset) * 0.1)
train_dataset = dataset[:one_tenth_length * 8]
val_dataset = dataset[one_tenth_length*8:one_tenth_length * 9]
test_dataset = dataset[one_tenth_length*9:]
len(train_dataset), len(val_dataset), len(test_dataset)

(40000, 5000, 5000)

In [26]:
# Load dataset into PyG loaders 
batch_size= 512
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [27]:
# Load dataset into PyG loaders 
num_items = df.item_id.max() +1
num_categories = df.category.max()+1
num_items , num_categories

(18461, 109)

### Настройка модели для обучения

In [28]:
embed_dim = 128
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Model Structure
        self.conv1 = GraphConv(embed_dim * 2, 128)
        self.pool1 = TopKPooling(128, ratio=0.9)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.9)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.9)
        self.item_embedding = torch.nn.Embedding(num_embeddings=num_items, embedding_dim=embed_dim)
        self.category_embedding = torch.nn.Embedding(num_embeddings=num_categories, embedding_dim=embed_dim)        
        self.lin1 = torch.nn.Linear(256, 256)
        self.lin2 = torch.nn.Linear(256, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    # Forward step of a model
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        item_id = x[:,:,0]
        category = x[:,:,1]
        

        emb_item = self.item_embedding(item_id).squeeze(1)
        emb_category = self.category_embedding(category).squeeze(1)
        
        x = torch.cat([emb_item, emb_category], dim=1)  
        # print(x.shape)
        x = F.relu(self.conv1(x, edge_index))
        # print(x.shape)
        r = self.pool1(x, edge_index, None, batch)
        # print(r)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.act2(x)      
        
        outputs = []
        for i in range(x.size(0)):
            output = torch.matmul(emb_item[data.batch == i], x[i,:])

            outputs.append(output)
              
        x = torch.cat(outputs, dim=0)
        x = torch.sigmoid(x)
        
        return x

### Обучение нейронной сверточной сети

In [62]:
# Enable CUDA computing
device = torch.device('cuda')
model = Net().to(device)

# Choose optimizer and criterion for learning
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
crit = torch.nn.BCELoss()
model
#old
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#crit = torch.nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.005)


Net(
  (conv1): GraphConv(256, 128)
  (pool1): TopKPooling(128, ratio=0.9, multiplier=1.0)
  (conv2): GraphConv(128, 128)
  (pool2): TopKPooling(128, ratio=0.9, multiplier=1.0)
  (conv3): GraphConv(128, 128)
  (pool3): TopKPooling(128, ratio=0.9, multiplier=1.0)
  (item_embedding): Embedding(18461, 128)
  (category_embedding): Embedding(109, 128)
  (lin1): Linear(in_features=256, out_features=256, bias=True)
  (lin2): Linear(in_features=256, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (act2): ReLU()
)

In [63]:
# Train function
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)

        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

In [64]:
# Evaluate result of a model
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    return roc_auc_score(labels, predictions)

In [68]:
# Train a model
NUM_EPOCHS =  10 #@param { type: "integer" }
for epoch in tqdm(range(NUM_EPOCHS)):
    loss = train()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

 10%|█         | 1/10 [00:37<05:34, 37.12s/it]

Epoch: 000, Loss: 0.60102, Train Auc: 0.52048, Val Auc: 0.51044, Test Auc: 0.52802


 20%|██        | 2/10 [01:14<04:56, 37.12s/it]

Epoch: 001, Loss: 0.56037, Train Auc: 0.52405, Val Auc: 0.51946, Test Auc: 0.53604


 30%|███       | 3/10 [01:51<04:19, 37.04s/it]

...

Epoch: 007, Loss: 0.41432, Train Auc: 0.57352, Val Auc: 0.54917, Test Auc: 0.55918


 90%|█████████ | 9/10 [05:32<00:36, 36.92s/it]

Epoch: 008, Loss: 0.39961, Train Auc: 0.57615, Val Auc: 0.54343, Test Auc: 0.55698


100%|██████████| 10/10 [06:09<00:00, 36.96s/it]

Epoch: 009, Loss: 0.38994, Train Auc: 0.58440, Val Auc: 0.55082, Test Auc: 0.56259


### Проверка результата с помощью примеров

In [69]:
# Подход №1 - из датасета
evaluate(DataLoader(test_dataset[40:60], batch_size=10))

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


0.7045454545454546

In [70]:
# Подход №2 - через создание сессии покупок
test_df = pd.DataFrame([
      [-1, 15219, 0],
      [-1, 15431, 0],
      [-1, 14371, 0],
      [-1, 15745, 0],
      [-2, 14594, 0],
      [-2, 16972, 11],
      [-2, 16943, 0],
      [-3, 17284, 0]
], columns=['session_id', 'item_id', 'category'])

test_data = transform_dataset(test_df, buy_item_dict)
test_data = DataLoader(test_data, batch_size=1)

with torch.no_grad():
    model.eval()
    for data in test_data:
        data = data.to(device)
        pred = model(data).detach().cpu().numpy()

        print(data, pred)

100%|██████████| 3/3 [00:00<00:00, 243.65it/s]

DataBatch(x=[1, 1, 2], edge_index=[2, 0], y=[1], batch=[1], ptr=[2]) [0.00021696]
DataBatch(x=[3, 1, 2], edge_index=[2, 2], y=[3], batch=[3], ptr=[2]) [0.07477905 0.18410347 0.16190793]
DataBatch(x=[4, 1, 2], edge_index=[2, 3], y=[4], batch=[4], ptr=[2]) [0.0461938  0.22504659 0.0109443  0.02072046]



/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
